# Demo 3: POI Clustering

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

import loci as lc
from loci import io
from loci import clustering
from loci import plots

## Create a POI GeoDataFrame from a remote CSV file produced by OSMWrangle

In [ ]:
bound = lc.io.retrieve_osm_loc('Athens, Greece', buffer_dist=10000)
remote_file = 'http://download.slipo.eu/results/osm-to-csv/europe/europe_greece-pois.osm.csv.zip'
pois = lc.io.import_osmwrangle(remote_file, bound=bound, target_crs='EPSG:2100')
pois.head()

## Compute clusters

In [ ]:
# use DBSCAN
alg = 'dbscan'
min_pts = 15
eps = 100

# use HDBSCAN
# alg = 'hdbscan'
# min_pts = 15
# eps = None

pois_in_clusters, eps_per_cluster = lc.clustering.compute_clusters(pois, alg=alg, min_pts=min_pts, eps=eps, n_jobs=-1)

## Show the POIs of a specific cluster

In [ ]:
selected_cluster = 2
selected_pois = pois_in_clusters.loc[pois_in_clusters['cluster_id'] == selected_cluster]
plots.map_points(selected_pois)

## Compute cluster shapes and show clusters on the map

In [ ]:
cluster_borders = lc.clustering.cluster_shapes(pois_in_clusters, 1, eps_per_cluster)
plots.map_choropleth(cluster_borders, id_field='cluster_id', value_field='size')

In [ ]:
cluster_borders = lc.clustering.cluster_shapes(pois_in_clusters, 2, eps_per_cluster)
plots.map_choropleth(cluster_borders, id_field='cluster_id', value_field='size')

In [ ]:
cluster_borders = lc.clustering.cluster_shapes(pois_in_clusters, 3, eps_per_cluster)
plots.map_choropleth(cluster_borders, id_field='cluster_id', value_field='size')

## Show buildings and streets in a specific cluster

In [ ]:
selected_cluster = cluster_borders.loc[cluster_borders['cluster_id'] == selected_cluster]
m = plots.map_cluster_contents_osm(selected_cluster)
m

## Compute AOIs for 'tourism' and 'restaurant' and compare them on the map

In [ ]:
pois_restaurant = lc.analytics.filter_by_kwd(pois, kwd_filter='restaurant')
pois_tourism = lc.analytics.filter_by_kwd(pois, kwd_filter='tourism')

alg = 'dbscan'
min_pts = 10
eps = 100

aois_restaurant, eps_rest = lc.clustering.compute_clusters(pois_restaurant, alg=alg, min_pts=min_pts, eps=eps, n_jobs=-1)
aois_tourism, eps_tour = lc.clustering.compute_clusters(pois_tourism, alg=alg, min_pts=min_pts, eps=eps, n_jobs=-1)

aois_restaurant = lc.clustering.cluster_shapes(aois_restaurant, 2, eps_rest)
aois_tourism = lc.clustering.cluster_shapes(aois_tourism, 2, eps_rest)

In [ ]:
plots.map_choropleth(aois_restaurant, id_field='cluster_id', value_field='size')

In [ ]:
plots.map_choropleth(aois_tourism, id_field='cluster_id', value_field='size')

In [ ]:
m = lc.plots.map_cluster_diff(aois_restaurant, aois_tourism)
m